# Pipeline Example

In [ ]:
import logging

from sudareph.data import Data
from sudareph.flow import Parallel
from sudareph.work import Work

logging.basicConfig(level=logging.DEBUG)

## Module Definition

In [ ]:
sum = Work('sum')
Summation = Work('Sum', prefix=str)


@sum.register
def _sum(a):
    return 'ASDF:' + a


@Summation.register
def _Sum(val, prefix: str) -> str:
    return prefix + val

# Level1: Use Module as function

In [ ]:
work = sum
result = work('INPUT')

print(result.output)

In [ ]:
work = Summation.set(prefix='ASDF:')
result = work('Input')

print(result.output)

## Level2: Use Module as smart function

In [ ]:
input = Data('name', 'value')
work = Summation.set(prefix='POST:')

result = input > work

print(result.output)

## Level3: Component can be joint

In [ ]:
pipeline = Summation.set(prefix='PRE:') >> Summation.set(prefix='POST:')
input = Data('name', 'value')

result = input > pipeline

print(result.output)

(Data('name', 'value') > Summation.set(prefix='PRE:') >> Summation.set(prefix='POST:')).output

## Parallel Workflow

In [ ]:
pipeline = Summation.set(prefix='Outer:') >> Parallel(
    'Parallel',
    A=Summation.set(prefix='Inner1') >> Summation.set(prefix='Inner2'),
    B=Summation.set(prefix='Inner1-1') >> Summation.set(prefix='Inner2-1'),
)

(Data('name', 'ARG') > pipeline).output

In [ ]:
pipeline = Summation.set(prefix='Outer:') >> Parallel(
    'Parallel',
    A=Parallel(
        'Parallel',
        A=Summation.set(prefix='Inner1') >> Summation.set(prefix='Inner2'),
        B=Summation.set(prefix='Inner1-1') >> Summation.set(prefix='Inner2-1'),
    ),
    B=Summation.set(prefix='Inner1-1') >> Summation.set(prefix='Inner2-1'),
)

(Data('name', 'ARG') > pipeline).output